In [1]:
import tensorflow as tf

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
import math, keras

In [12]:
TRAIN_DATA_DIR = 'train'
VALIDATION_DATA_DIR = 'val'
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

In [13]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [23]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=12345,
    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [15]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [24]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=['acc'])

num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)

model.fit(train_generator,
    steps_per_epoch = num_steps,
    epochs=10,
    validation_data = validation_generator,
    validation_steps = num_steps)

Epoch 1/10
8/8 [==============================] - 10s 1s/step - loss: 0.5706 - acc: 0.7080 - val_loss: 0.1902 - val_acc: 0.9420
Epoch 2/10
8/8 [==============================] - 9s 1s/step - loss: 0.2088 - acc: 0.9140 - val_loss: 0.1039 - val_acc: 0.9660
Epoch 3/10
8/8 [==============================] - 8s 1s/step - loss: 0.1481 - acc: 0.9480 - val_loss: 0.0837 - val_acc: 0.9740
Epoch 4/10
8/8 [==============================] - 8s 1s/step - loss: 0.1202 - acc: 0.9560 - val_loss: 0.0801 - val_acc: 0.9720
Epoch 5/10
8/8 [==============================] - 8s 1s/step - loss: 0.0893 - acc: 0.9700 - val_loss: 0.0724 - val_acc: 0.9800
Epoch 6/10
8/8 [==============================] - 8s 1s/step - loss: 0.0771 - acc: 0.9700 - val_loss: 0.0681 - val_acc: 0.9800
Epoch 7/10
8/8 [==============================] - 8s 1s/step - loss: 0.0581 - acc: 0.9760 - val_loss: 0.0691 - val_acc: 0.9780
Epoch 8/10
8/8 [==============================] - 8s 1s/step - loss: 0.0586 - acc: 0.9820 - val_loss: 0.0645 -

In [25]:
model.save('model.h5')

C:\Users\Максимка\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
model.save('model.keras')